# Algoritmos de optimización - Seminario

#### Nico Toscano

#### URL: https://github.com/NicolaiOcin/03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEM           

#### Problema 2. Organizar los horarios de partidos de La Liga
• En primer lugar se clasifican los equipos en tres categorías según el numero de
seguidores( que tiene una relación directa con la audiencia). Hay 4 equipos en la
categoría A, 10 equipos de categoría B y 6 equipos de categoría C.<br>
• Se conoce estadísticamente la audiencia que genera cada partido según los equipos
que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)<br>
• Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce
según los coeficientes de la siguiente tabla<br>
• Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes<br>
• Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido
se verá afectada y se estima que se reduce en porcentaje según la siguiente tabla
dependiendo del número de coincidencias:

In [121]:
# Librerias
import random
import copy
import itertools
from time import time
from functools import wraps

def calcular_tiempo(f):
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time()
        salida = f(*args, **kwargs)
        t_final = time()
        print('Tiempo transcurrido (en segundos): {}'.format(t_final - t_inicial))
        return salida
    return cronometro

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

> Sin tener en cuenta las restricciones, las posibilidades son todas las posibles permutaciones de colocar los 10 equipos en los 10 horarios. Por lo tanto es factorial de 10 (10!) que es igual a 3628800 posibilidades.

> Teniendo en cuenta la repetición de horarios, el número de posibilidades seria de 10^10, teniendo en cuenta que tenemos que restar las combinaciones, en que no aparecen los partidos en un lunes y un viernes.

In [122]:
# Reptiendo horariso podriamos calcular todas las combinaciones de la siguiente manera (por cumpoto hago que se ejecute)
#comb_rep_horarios = list(itertools.product([0,1,2,3,4,5,6,7,8,9],repeat = 6))

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

> Ramificación y poda porque podemos crear un árbol con las distintas soluciones posibles, y para no recorrer todos los nodos, utilizamos este método. Y de esta manera encontrar la solución exacta. Podemos utilizar métodos heurísticos o algoritmos genéticos pero la solución estará cerca de la exacta.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

> La función objetivo es el número de espectadores que obtenemos dependiendo de como colocamos los partidos.

> Es un problema de maximización debido a que se nos pide colocar los partidos de tal manera que obtengamos los maximos espectadores posibles.

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

> ---Este algoritmo se encuentra al final del documento "Problema_2_alg_gen"---

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

> La complejidad del algoritmo por fuerza bruta es de O(n!), factorial.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [123]:
# Equipos
Equipos = {'A':['Eq1','Eq2','Eq3','Eq4'], 'B':['Eq5','Eq6','Eq7','Eq8','Eq9','Eq10','Eq11','Eq12','Eq13','Eq14'], 
      'C':['Eq15','Eq16','Eq17','Eq18','Eq19','Eq20']}

Equipos_lista = ['Eq1','Eq2','Eq3','Eq4','Eq5','Eq6','Eq7','Eq8','Eq9','Eq10','Eq11','Eq12','Eq13','Eq14','Eq15',
                'Eq16','Eq17','Eq18','Eq19','Eq20']

In [124]:
# Generar una jornada aleatoria (combinacion de equipos)
def generar_jornada():
    j = []
    Equipos_lista_copy = copy.deepcopy(Equipos_lista)
    for i in range(10):
        x = random.sample(Equipos_lista_copy,2)
        j.append(x)
        Equipos_lista_copy.remove(x[0])
        Equipos_lista_copy.remove(x[1])
    return j

In [125]:
# Coeficientes segun las categorias de los equipos
def cat(E1,E2):
    if E1 in Equipos['A'] and E2 in Equipos['A']:
        return 2
    if E1 in Equipos['A'] and E2 in Equipos['B']:
        return 1.3
    if E1 in Equipos['A'] and E2 in Equipos['C']:
        return 1
    if E1 in Equipos['B'] and E2 in Equipos['A']:
        return 1.3 
    if E1 in Equipos['B'] and E2 in Equipos['B']:
        return 0.9  
    if E1 in Equipos['B'] and E2 in Equipos['C']:
        return 0.75
    if E1 in Equipos['C'] and E2 in Equipos['A']:
        return 1
    if E1 in Equipos['C'] and E2 in Equipos['B']:
        return 0.75
    if E1 in Equipos['C'] and E2 in Equipos['C']:
        return 0.47

In [126]:
# Coeficientes segun si hay coincidencias de partidos
def coef_c(n):
    if n == 0:
             c = 1
    else:
        if n == 1:
            c = 1-0.25
        if n == 2:
            c = 1-0.45
        if n == 3:
            c = 1-0.6
        if n == 4:
            c = 1-0.7
        if n == 5:
            c = 1-0.75
        if n == 6:
            c = 1-0.78
        if n >= 7:
            c = 1-0.8
    return c

In [127]:
# Valor de la solucion
def valor(solucion,costes_mod):
    valor = 0
    rep = []
    coef = []
    
    for i in solucion:
        rep.append(solucion.count(i)-1)
    
    for i in rep:
        coef.append(coef_c(i))
    
    for i in range(len(solucion)):
        x = costes_mod[solucion[i]][i]
        x = x*coef[i]
        valor += x
        
    return valor

In [128]:
# Valor del partido segun la categoria de cada equipo
def jornada_valor(jornada):
    result = []
    for i in jornada:
        result.append(cat(i[0],i[1]))
    return result

In [129]:
# Modificación de la matriz de costes segun "jornada valor"
def mod_costes(jornada_valor,costes):
    N = len(costes)
    result = [[0]*N for i in [0]*N]
    for i in range(len(jornada_valor)):
        for j in range(len(costes)):
            result[i][j] = round(jornada_valor[j]*costes[i][j],3)
    return result

In [130]:
# Crear hijos (nodos)
def crear_hijos(nodo,N):
    hijos = []
    elem = []
    for i in range(N):
        if len(nodo)>=8:
            x_0= nodo.count(0)
            x_9= nodo.count(9)
            if x_0 == 0 and 0 not in elem:
                hijos.append({'s':nodo+(0,)})
                elem.append(0)
            if x_9 == 0 and 9 not in elem:
                hijos.append({'s':nodo+(9,)})
                elem.append(9)
            if x_0 !=0 and x_9 != 0:
                hijos.append({'s':nodo+(i,)})
        else:
            hijos.append({'s':nodo+(i,)})
    return hijos   

In [131]:
# Coste Inferior (de los nodos)
def CI(s,costes_mod):
    valor_acum = 0
    valor_temp = 0
    valor_parcial = 0
    valor_parcial_ref = 0
    posiciones = []
    pos_final = []
    pos_temp = []
    if len(s)>0:
        valor_acum += valor(s,costes_mod)
        for i in s:
            pos_temp.append(i)
            pos_final.append(i)
    valor_parcial += valor_acum      
    ls = len(s)
    lc = len(costes_mod)
    for i in range(ls,lc):
        valor_ref = 0
        for j in range(len(costes_mod)):
            rep = posiciones.count(j)
            if j in s:
                rep = s.count(j)
                x = costes[j][i]
                x = x*coef_c(rep)
                valor_temp_2 = valor_parcial+x
                if valor_temp_2> valor_parcial_ref:
                    valor_parcial_ref = valor_temp_2
                    valor_ref = x
                    pos = j
            else:
                pos_temp.append(j)
                valor_temp = valor(pos_temp,costes_mod)
                
                if valor_temp> valor_ref:
                    valor_ref = valor_temp
                    pos = j
                pos_temp.remove(j)
        pos_temp.append(pos)
        pos_final.append(pos)
        valor_parcial_ref = valor(pos_temp,costes_mod)
    valor_acum = valor(pos_final,costes_mod)
    return round(valor_acum,3)

In [132]:
# Ramificación y poda
@calcular_tiempo
def ramificacion_y_poda(COSTES):
    DIMENSION = len(COSTES)
    MEJOR_SOLUCION = tuple(i for i in range(DIMENSION))
    CotaSup = valor(MEJOR_SOLUCION, COSTES)
    
    #print("Cota Superior:", CotaSup)
    
    NODOS = []
    
    NODOS.append({'s':(), 'ci':CI((), COSTES)})
    
    #print(NODOS)
    
    iteracion = 0
    
    while (len(NODOS) > 0):
        iteracion += 1
        #print('\n#', iteracion)
        
        nodo_prometedor = max(NODOS, key=lambda x: x['ci'])
        #print("Nodo prometedor:", nodo_prometedor)
        
        # Ramificación (se generan los HIJOS)
        HIJOS = [{'s': x['s'], 'ci':CI(x['s'], COSTES)} for x in crear_hijos(nodo_prometedor['s'], DIMENSION)]
     
        # Revisamos la cota superior y nos quedamos con la mejor solución si llegamos a una solución final
        NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION]
       
        if len(NODO_FINAL) > 0:
            #print("\n********Soluciones:", [x for x in HIJOS if len(x['s']) == DIMENSION])
            
            for i in NODO_FINAL:
                if i['ci'] > CotaSup:
                    CotaSup = i['ci']
                    MEJOR_SOLUCION = i
                    
        #Poda
        HIJOS = [x for x in HIJOS if x['ci']>CotaSup]
        
        #Eliminamos el nodo expandido
        NODOS = [x for x in NODOS if x['s'] != nodo_prometedor['s']]
        
        #Añadimos los hijos
        NODOS.extend(HIJOS)
        
        #print(HIJOS)
        
    print("La mejor solucion es:", MEJOR_SOLUCION)

In [133]:
# Matriz de costes
# Las columnas representan los diferentes partidos, y las filas los distintos horarios donde se puede asignar un partido
# Los valores representan los coeficientes que suponen colocar un partido (columna) en ese horario (fila)
costes = [[0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4],
         [0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.55,],
          [0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,],
          [0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,],
          [1,1,1,1,1,1,1,1,1,1,],
          [0.45,0.45,0.45,0.45,0.45,0.45,0.45,0.45,0.45,0.45,],
          [0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,],
          [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,],
          [1,1,1,1,1,1,1,1,1,1,],
          [0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,]]      

In [143]:
# Generamos una jornada aleatoria
jornada = generar_jornada()
jornada

[['Eq17', 'Eq7'],
 ['Eq5', 'Eq20'],
 ['Eq12', 'Eq9'],
 ['Eq13', 'Eq2'],
 ['Eq3', 'Eq14'],
 ['Eq18', 'Eq11'],
 ['Eq10', 'Eq19'],
 ['Eq16', 'Eq1'],
 ['Eq15', 'Eq8'],
 ['Eq6', 'Eq4']]

In [144]:
# Miramos el valor de cada partido, segun la categoria de cada equipo
jornada_val = jornada_valor(jornada)
jornada_val

[0.75, 0.75, 0.9, 1.3, 1.3, 0.75, 0.75, 1, 0.75, 1.3]

In [145]:
# Modificamos la matriz de costes, segun los valores de la jornada que tenemos
costes_mod = mod_costes(jornada_val,costes)
costes_mod

[[0.3, 0.3, 0.36, 0.52, 0.52, 0.3, 0.3, 0.4, 0.3, 0.52],
 [0.413, 0.413, 0.495, 0.715, 0.715, 0.413, 0.413, 0.55, 0.413, 0.715],
 [0.525, 0.525, 0.63, 0.91, 0.91, 0.525, 0.525, 0.7, 0.525, 0.91],
 [0.6, 0.6, 0.72, 1.04, 1.04, 0.6, 0.6, 0.8, 0.6, 1.04],
 [0.75, 0.75, 0.9, 1.3, 1.3, 0.75, 0.75, 1, 0.75, 1.3],
 [0.338, 0.338, 0.405, 0.585, 0.585, 0.338, 0.338, 0.45, 0.338, 0.585],
 [0.562, 0.562, 0.675, 0.975, 0.975, 0.562, 0.562, 0.75, 0.562, 0.975],
 [0.375, 0.375, 0.45, 0.65, 0.65, 0.375, 0.375, 0.5, 0.375, 0.65],
 [0.75, 0.75, 0.9, 1.3, 1.3, 0.75, 0.75, 1, 0.75, 1.3],
 [0.3, 0.3, 0.36, 0.52, 0.52, 0.3, 0.3, 0.4, 0.3, 0.52]]

In [146]:
# Resultado con Ramificación y poda
ramificacion_y_poda(costes_mod)

La mejor solucion es: {'s': (5, 7, 2, 3, 4, 0, 1, 6, 9, 8), 'ci': 6.746}
Tiempo transcurrido (en segundos): 0.1905210018157959


In [147]:
# Juntamos los pasos para generar automaticamente una solución para cualquier jornada
def jornada_sol_auto(jornada):
    jornada_val = jornada_valor(jornada)
    costes_mod = mod_costes(jornada_val,costes)
    resultado = ramificacion_y_poda(costes_mod)
    return resultado

In [148]:
# Resultado
jornada_sol_auto(jornada)

La mejor solucion es: {'s': (5, 7, 2, 3, 4, 0, 1, 6, 9, 8), 'ci': 6.746}
Tiempo transcurrido (en segundos): 0.18952417373657227


(*)Calcula la complejidad del algoritmo 

Respuesta

> La complejidad de este algoritmo es casi lineal O(nlogn).

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [149]:
# Conjunto de jornadas
def conjunto_jornadas(N):
    jornadas = []
    for _ in range(N):
        jornadas.append(generar_jornada())
    return jornadas

In [150]:
# Conjunto de 10 jornadas
jornadas = conjunto_jornadas(5)
jornadas

[[['Eq20', 'Eq19'],
  ['Eq17', 'Eq9'],
  ['Eq3', 'Eq11'],
  ['Eq8', 'Eq18'],
  ['Eq1', 'Eq6'],
  ['Eq7', 'Eq5'],
  ['Eq4', 'Eq12'],
  ['Eq14', 'Eq2'],
  ['Eq13', 'Eq16'],
  ['Eq15', 'Eq10']],
 [['Eq1', 'Eq14'],
  ['Eq6', 'Eq2'],
  ['Eq8', 'Eq3'],
  ['Eq16', 'Eq18'],
  ['Eq5', 'Eq10'],
  ['Eq13', 'Eq9'],
  ['Eq12', 'Eq11'],
  ['Eq20', 'Eq7'],
  ['Eq17', 'Eq4'],
  ['Eq15', 'Eq19']],
 [['Eq20', 'Eq19'],
  ['Eq2', 'Eq3'],
  ['Eq14', 'Eq7'],
  ['Eq13', 'Eq11'],
  ['Eq8', 'Eq9'],
  ['Eq15', 'Eq10'],
  ['Eq5', 'Eq18'],
  ['Eq17', 'Eq4'],
  ['Eq12', 'Eq1'],
  ['Eq6', 'Eq16']],
 [['Eq2', 'Eq6'],
  ['Eq13', 'Eq16'],
  ['Eq15', 'Eq4'],
  ['Eq1', 'Eq9'],
  ['Eq17', 'Eq10'],
  ['Eq14', 'Eq3'],
  ['Eq19', 'Eq12'],
  ['Eq20', 'Eq8'],
  ['Eq5', 'Eq11'],
  ['Eq7', 'Eq18']],
 [['Eq5', 'Eq2'],
  ['Eq18', 'Eq4'],
  ['Eq16', 'Eq13'],
  ['Eq6', 'Eq14'],
  ['Eq8', 'Eq3'],
  ['Eq20', 'Eq17'],
  ['Eq9', 'Eq7'],
  ['Eq15', 'Eq1'],
  ['Eq12', 'Eq11'],
  ['Eq10', 'Eq19']]]

Aplica el algoritmo al juego de datos generado

Respuesta

In [151]:
res = []
for i in jornadas:
    res.append(jornada_sol_auto(i)) 

La mejor solucion es: {'s': (7, 5, 2, 9, 3, 1, 4, 8, 0, 6), 'ci': 6.78}
Tiempo transcurrido (en segundos): 0.2403888702392578
La mejor solucion es: {'s': (3, 6, 4, 9, 1, 2, 7, 0, 8, 5), 'ci': 6.589}
Tiempo transcurrido (en segundos): 0.6602368354797363
La mejor solucion es: {'s': (7, 4, 5, 1, 6, 9, 0, 3, 8, 2), 'ci': 7.035}
Tiempo transcurrido (en segundos): 0.1725330352783203
La mejor solucion es: {'s': (3, 1, 2, 6, 5, 4, 0, 9, 8, 7), 'ci': 6.641}
Tiempo transcurrido (en segundos): 0.11966705322265625
La mejor solucion es: {'s': (3, 1, 5, 7, 4, 0, 2, 8, 9, 6), 'ci': 6.418}
Tiempo transcurrido (en segundos): 0.11070036888122559


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

> Cualquier resultado de Google que me resolviera las dudas que tenia a la hora de aplicar una función.

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

> · Se podrian añadir más categorias, más horarios, más equipos (o quitar), y tambíen se podrían añadir más restricciones, cómo la coincidencia de horarios, de otros partidos de otras ligas (como la liga inglesa, italiana, alemana, francesa,etc.)<br><br>
> · La coincidencia de horarios en vez de reducir a todos los equipos con el mismo coeficiente, reducirlo con un coeficiente que dependiera de las categorias de los equipos, de tal manera que los equipos con menos categoria tubieran un coeficiente de reducción más elevado que equipos de mayor categoria. Porque la realidad es que un barça-madrid si se solapa con otros partidos de categorias inferiores, no se reducen por igual todos los partidos solapados, el número de espectadores.